In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [2]:
# 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-17 20:19:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.5’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-17 20:19:53 (5.93 MB/s) - ‘postgresql-42.2.9.jar.5’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

watch_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
watch_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

# Total Number of Records #

In [5]:
total_rows = watch_df.count()
print(f"The total number of records in the dataset is {total_rows}")

The total number of records in the dataset is 960872


In [6]:
watch_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# create the review id table
watch_review_id_temp_df = watch_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
watch_review_id_temp_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-08-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-08-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-08

In [8]:
# check schema
watch_review_id_temp_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# convert review_date to DATE from string
from pyspark.sql.functions import to_date
watch_review_id_df = watch_review_id_temp_df.withColumn('REVIEW_DATED', to_date(watch_review_id_temp_df.review_date, 'yyyy-MM-dd'))
watch_review_id_df.printSchema()



root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)
 |-- REVIEW_DATED: date (nullable = true)



In [10]:
# drop "review_date" column
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
watch_review_id_df = watch_review_id_df.drop("review_date")
watch_review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- REVIEW_DATED: date (nullable = true)



In [11]:
# rename "REVIEW_DATE" column
watch_review_id_df = watch_review_id_df.withColumnRenamed('REVIEW_DATED', 'review_dated')
watch_review_id_df.show()

+--------------+-----------+----------+--------------+------------+
|     review_id|customer_id|product_id|product_parent|review_dated|
+--------------+-----------+----------+--------------+------------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370|  2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722|  2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390|  2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625|  2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221|  2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695|  2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279|  2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113|  2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892|  2015-08-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407|  2015-08-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857|  2015-08-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     6854

In [12]:
# create the products table df
watch_products_df = watch_df.select(['product_id', "product_title"])
watch_products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [13]:
# create the customers df
from pyspark.sql.functions import desc
watch_customers_df = watch_df.select(['customer_id'])\
  .groupby("customer_id")\
  .agg({"customer_id": "count"})\
  .orderBy(desc("count(customer_id)"))
watch_customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   42418272|               407|
|   40765068|               127|
|   19619134|               121|
|   52460875|               111|
|    5072809|               106|
|   18691646|                92|
|   14726863|                88|
|   13355404|                87|
|   42416004|                86|
|   45112699|                75|
|   44191290|                70|
|   22962882|                66|
|   47769852|                65|
|   23137720|                64|
|   25160623|                59|
|   21375608|                59|
|    5956842|                59|
|    8887467|                58|
|   24254971|                58|
|    1389173|                58|
+-----------+------------------+
only showing top 20 rows



In [14]:
# create the vine df
watch_vine_df = watch_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
watch_vine_df.show()



+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
|R15C7QEZT0LGZN|          4|            2|          2|   N|
|R361XSS37V0NCZ|          1|            0|          0|   N|
| ROTNLALUAJAUB|          3|            0|          0|   N|
|R2DYX7QU6BGOHR|          5|            

### postgres setup ###

In [21]:
# configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonpostgresdb.cec7i89ma4zy.us-east-2.rds.amazonaws.com:5432/amazon_db"
config = {"user":"xxxxxxxxxx", 
          "password": "xxxxxxxxxx",", 
          "driver":"org.postgresql.Driver"}

In [22]:
# write the watch_review_id_df to the watch_review_id table in postress

watch_review_id_df.write.jdbc(url=jdbc_url, table='watch_review_id', mode=mode, properties=config)

Py4JJavaError: ignored